In [3]:
import pyprind # https://anaconda.org/conda-forge/pyprind
import pandas as pd
import os

basepath = '/Users/nagaitakeyuki/Documents/00.my-work/20191229_python_ml_programming/8章/aclImdb'

labels = {'pos': 1, 'neg':0}
pbar = pyprind.ProgBar(50000)

df = pd.DataFrame()

for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()

df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:03:00


In [4]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [5]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [6]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array(['The sun is shining', 
                             'The weather is seet', 
                             'The sun is shining, the weager is weet, and one and one is two'])
bag = count.fit_transform(docs)

In [11]:
print(count.vocabulary_)
print(count.get_feature_names())

{'the': 6, 'sun': 5, 'is': 1, 'shining': 4, 'weather': 9, 'seet': 3, 'weager': 8, 'weet': 10, 'and': 0, 'one': 2, 'two': 7}
['and', 'is', 'one', 'seet', 'shining', 'sun', 'the', 'two', 'weager', 'weather', 'weet']


In [13]:
print(bag.toarray())

# 例えば、(0, 6)	1　だと、0番目の文には、インデックス6の単語であるtheが　1回 出現している、という意味。
print(bag)

[[0 1 0 0 1 1 1 0 0 0 0]
 [0 1 0 1 0 0 1 0 0 1 0]
 [2 3 2 0 1 1 2 1 1 0 1]]
  (0, 6)	1
  (0, 5)	1
  (0, 1)	1
  (0, 4)	1
  (1, 6)	1
  (1, 1)	1
  (1, 9)	1
  (1, 3)	1
  (2, 6)	2
  (2, 5)	1
  (2, 1)	3
  (2, 4)	1
  (2, 8)	1
  (2, 10)	1
  (2, 0)	2
  (2, 2)	2
  (2, 7)	1


In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.   0.56 0.56 0.43 0.   0.   0.   0.  ]
 [0.   0.36 0.   0.61 0.   0.   0.36 0.   0.   0.61 0.  ]
 [0.49 0.43 0.49 0.   0.19 0.19 0.29 0.24 0.24 0.   0.24]]


In [14]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [24]:
import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    
    # :, ;, =  のいずれか　　　＋　　　- あってもなくてもいい　　　＋　　　 ), (, D, P　　　のいずれか 　　　  ->  例えば、:-)  のような顔文字
    # (?:)　は()のメモリ節約バージョン。　　後半の\)|\(は、)と(をエスケープしてるだけ。
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    
    # 全文字を小文字に変換し、英数字、アンダースコア以外の文字を削除。
    # 顔文字から鼻を表す-を消して、お尻につけている。
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    
    return text 

In [25]:
preprocessor('<h1>aaDDaa</h1>  :-)  :D  hoge ')

'aaddaa d hoge :) :D'

In [26]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [28]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [29]:
df['review'] = df['review'].apply(preprocessor)

In [32]:
df.loc[0, 'review'][-50:]

'zation my vote is seven title brazil not available'

In [34]:
def tokenizer(text):
    return text.split()

tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [35]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [36]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nagaitakeyuki/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [37]:
from nltk.corpus import stopwords

stop = stopwords.words('english')

[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:]
        if w not in stop]


['runner', 'like', 'run', 'run', 'lot']

In [38]:
[w for w in tokenizer_porter('a runner likes running and runs a lot')
        if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

In [39]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values

X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [42]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
                           'vect__stop_words': [stop, None],
                           'vect__tokenizer': [tokenizer, tokenizer_porter],
                           'clf__penalty': ['l1', 'l2'],
                           'clf__C': [1.0, 10.0, 100.0]},
                          {'vect__ngram_range': [(1, 1)],
                           'vect__stop_words': [stop, None],
                           'vect__tokenizer': [tokenizer, tokenizer_porter],
                           'vect__use_idf': [False],
                           'vect__norm': [None],
                           'clf__penalty': ['l1', 'l2'],
                           'clf__C': [1.0, 10.0, 100.0]}]

lr_tfidf = Pipeline([('vect', tfidf),
                                ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)

gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 26.2min
/opt/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


KeyboardInterrupt: 

In [43]:
import numpy as np
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    
    return tokenized

In [44]:
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) # ヘッダーを読み飛ばす
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label        

In [46]:
next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [48]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    
    return docs, y

In [49]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', n_features=2**21, preprocessor=None, tokenizer=tokenizer)

clf = SGDClassifier(loss='log', random_state=1, max_iter=1)

doc_stream = stream_docs(path='movie_data.csv')

In [50]:
import pyprind

pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])

for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:26


In [51]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.868


In [52]:
clf = clf.partial_fit(X_test, y_test)

In [53]:
import pandas as pd
df = pd.read_csv('movie_data.csv', encoding='utf-8')

In [55]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english', max_df=.1, max_features=5000)
X = count.fit_transform(df['review'].values)

In [57]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, random_state=123, learning_method='batch')

X_topics = lda.fit_transform(X)

In [58]:
lda.components_.shape

(10, 5000)

In [61]:
n_top_words = 5
feature_names = count.get_feature_names()

for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d" % (topic_idx + 1))
    
    # topicには、単語ごとの重要度の数値が格納されている。単語数は5000個。
    # argsort()で、それを昇順に並びかえる。結果には、実際の数値ではなく、もともとのtopicの中での位置（インデックス）が格納されている。
    #  例えば、[3, 1, 2] -> 昇順に並びかえる -> [1, 2, 3] -> もとの位置を格納 -> [1, 2, 0]
    # 
    # もともとのtopicの位置（インデックス）は、単語名が格納されているfeature_namesでの位置（インデックス）と、一致している。
    # なので、feature_names[i]とすることで、対応する単語名が取得できる。
    print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words -1:-1]]))

Topic 1
worst minutes awful script stupid
Topic 2
family mother father children girl
Topic 3
american war dvd music tv
Topic 4
human audience cinema art sense
Topic 5
police guy car dead murder
Topic 6
horror house sex girl woman
Topic 7
role performance comedy actor performances
Topic 8
series episode war episodes tv
Topic 9
book version original read novel
Topic 10
action fight guy guys cool


In [66]:
horror = X_topics[:, 5].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\nHorror movie #%d: ' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')


Horror movie #1: 
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...

Horror movie #2: 
Okay, what the hell kind of TRASH have I been watching now? "The Witches' Mountain" has got to be one of the most incoherent and insane Spanish exploitation flicks ever and yet, at the same time, it's also strangely compelling. There's absolutely nothing that makes sense here and I even doubt there  ...

Horror movie #3: 
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...


In [65]:
# 文書ごとに、各トピックに属する確率が格納されている模様。

print(X_topics.shape)
print(X_topics)
print(np.argmax(X_topics, axis=1))

(50000, 10)
[[0.   0.2  0.   ... 0.   0.   0.  ]
 [0.99 0.   0.   ... 0.   0.   0.  ]
 [0.39 0.43 0.11 ... 0.   0.   0.  ]
 ...
 [0.45 0.   0.   ... 0.   0.1  0.1 ]
 [0.   0.53 0.   ... 0.   0.   0.  ]
 [0.01 0.48 0.01 ... 0.01 0.01 0.45]]
[4 0 1 ... 0 1 1]


In [67]:
import pickle
import os

dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

pickle.dump(stop,
                        open(os.path.join(dest, 'stopwords.pkl'), 'wb'),
                       protocol=4)

pickle.dump(clf,
                       open(os.path.join(dest, 'classifier.pkl'), 'wb'),
                       protocol=4)